# Question
The following figures show the Di Wang Tower in Shenzhen. The structure of Di Wang Tower is made up of steel and reinforced concrete (calculated as reinforced concrete), C terrain type, the design wind pressure $w_0 = 0.75 \text{ kN/m}^2$, period $T_1=6.38\text{ s}$, The first mode shape is listed in following table. To determine shape factor, the cross section could be considered as rectangle.

<center>
    <img src="./images/Diwang01.png" alt="The Elvation of the DiWang Tower" width="400"/>
    <img src="./images/Diwang02.png" alt="The Plan View of the DiWang Tower" width="400"/>
</center>

<br>

| $\dfrac{h}{H}$ | 0     | 0.1   | 0.2   | 0.3   | 0.4    | 0.5   | 0.6   | 0.7    | 0.8   | 0.9   | 1.0   |
| -------------- | ----- | ----- | ----- | ----- | ------ | ----- | ----- | ------ | ----- | ----- | ----- |
| Mode shape     | 0.0000| 0.0137| 0.0558| 0.1277| 0.2424| 0.3503| 0.4629| 0.5903| 0.7309| 0.8700| 1.0000| 
| Mass / ton     | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 1500  | 

Only consider the first mode, calculate **the extreme of top displacements** induced by the along-wind static and dynamic wind load in the given wind direction using the following methods:
1. Method recommended by the Chinese code for wind-resistant design of buildings (GB 50009-2012).
2. According to the quasi-steady assumption, convert the fluctuating wind velocity time histories in `windData` (**with a duration of 10 minutes and a sampling frequency of 10 Hz**) to wind load time histories $F_i(t)=[1/2\rho \bar{u_i}^2+\rho \bar{u_i} u'_i(t)] \cdot \mu_{s,i} \cdot A_i$, and then using stocastic vibration methods in time domain (such as the newmark-beta algorithm) and in frequency domain (such as the response spectrum analysis). Peak factor $g = 2.5$ should be used for the extreme value calculation, and the programming using frequency domain method is optional.

# Answer


In [3]:
import scipy.io as sio

# load data
simDataPath = './windData/windData.mat'
simData = sio.loadmat(simDataPath, squeeze_me=True, struct_as_record=False)
U = simData['U']
Z = simData['Z']
dt = simData['dt']
t = simData['t']
del simData
print("\nThe dimension of U is:", U.shape)


The dimension of U is: (11, 6000)


In [2]:
import pandas as pd
import numpy as np

# 表格数据
data = {
    'i': list(range(11)),
    'z_i':[ 0,32.495,64.99,97.485,129.98,162.475,194.97,227.465,259.96,292.455,324.95],
    'phi_i': [0.000, 0.0137, 0.0558, 0.1277, 0.2424, 0.3503, 0.4629, 0.5903, 0.7309, 0.8700, 1.0000],
    'm_i': [2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 1500]
}

# 计算标准风压 w_ki
def calculate_wki(z_i, phi_i):
    return 3.355 * ((z_i / 450) ** 0.44 + 0.433 * phi_i)

# 计算模态质量 M
def calculate_M(m_i, phi_i):
    return sum(m_i * np.array(phi_i) ** 2)

# 计算集中力 f_i
def calculate_fi(w_ki, i):
    base_fi = 65.88 * 30.346 * w_ki
    if i == 1:
        return 1.5 * base_fi
    elif i == 10:
        return 0.5 * base_fi
    else:
        return base_fi

# 计算模态力 F
def calculate_F(phi_i, f_i):
    return sum(np.array(phi_i) * np.array(f_i))

# 计算模态坐标 η
def calculate_eta(F, M):
    return F / (0.970 * M)

# 计算位移响应 X_i
def calculate_Xi(eta, phi_i):
    return eta * np.array(phi_i)

# 计算 w_ki
data['w_ki'] = [calculate_wki(z, phi) for z, phi in zip(data['z_i'], data['phi_i'])]

# 计算 M
M = calculate_M(data['m_i'], data['phi_i'])

# 计算 f_i
data['f_i'] = [calculate_fi(w, i) for w, i in zip(data['w_ki'], data['i'])]

# 计算 F
F = calculate_F(data['phi_i'], data['f_i'])

# 计算 η
eta = calculate_eta(F, M)

# 计算 X_i
data['X_i'] = calculate_Xi(eta, data['phi_i'])

# 创建 DataFrame
df = pd.DataFrame(data)

# 输出结果
print("计算结果：")
print(f"模态质量 M: {M:.2f}")
print(f"模态力 F: {F:.2f}")
print(f"模态坐标 η: {eta:.4f}")
print(df[['i', 'w_ki', 'f_i', 'X_i']])

计算结果：
模态质量 M: 6637.30
模态力 F: 27168.45
模态坐标 η: 4.2199
     i      w_ki          f_i       X_i
0    0  0.000000     0.000000  0.000000
1    1  1.075451  3225.053397  0.057813
2    2  1.513024  3024.828233  0.235470
3    3  1.897149  3792.769084  0.538880
4    4  2.294744  4587.639475  1.022902
5    5  2.651901  5301.664861  1.478228
6    6  2.994476  5986.539575  1.953388
7    7  3.342509  6682.326123  2.491002
8    8  3.697137  7391.295681  3.084319
9    9  4.039386  8075.518469  3.671306
10  10  4.359938  4358.181662  4.219892


In [ ]:
1.根据GB 50009-2012算出最大位移量为4.220m

In [5]:
import numpy as np
import scipy.io as sio

# 常量定义
rho = 1.225  # 空气密度 (kg/m^3)
A_i = 1.0  # 假设的面积 (m^2)
E = 210e9  # 弹性模量 (Pa)
I = 245616  # 惯性矩 (m^4)
S = 2338.74  # 截面积 (m^2)
q = 34.87  # 建筑的密度 (kg/m^3)
g = 2.5  # 峰值因子
total_height = 324.95  # 总高度 (m)
num_nodes = 11
dt = 0.1  # 时间步长 (s)
total_time = 600  # 总时间 (s)
damping_ratio = 0.05  # 阻尼系数

# 加载风速数据
simDataPath = './windData/windData.mat'
simData = sio.loadmat(simDataPath, squeeze_me=True, struct_as_record=False)
U = simData['U']  # 风速时间历程
Z = simData['Z']
del simData

# 确保 mu_s_i 的长度与 U 的列数一致
mu_s_i = [1070.3853] + [2140.7706] * (U.shape[1] - 2) + [1070.3853]

# 计算风荷载时间历程 F_i(t)
def calculate_wind_load(U, rho, mu_s_i, A_i):
    u_bar = np.mean(U, axis=0)  # 平均风速
    u_prime = U - u_bar  # 风速波动分量
    F_i_t = np.array([(0.5 * rho * u_bar[i]**2 + rho * u_bar[i] * u_prime[:, i]) * mu_s_i[i] * A_i for i in range(U.shape[1])]).T
    return F_i_t

# 使用Newmark-beta算法计算位移，考虑阻尼
def newmark_beta_damped(F_i_t, dt, total_time, damping_ratio, beta=0.25, gamma=0.5):
    num_steps = F_i_t.shape[0]  # 使用 F_i_t 的时间步数
    num_dofs = F_i_t.shape[1]
    u = np.zeros((num_steps, num_dofs))
    v = np.zeros((num_steps, num_dofs))
    a = np.zeros((num_steps, num_dofs))
    
    # 初始化
    u[0, :] = 0
    v[0, :] = 0
    a[0, :] = F_i_t[0, :] / (q * S)  # 初始加速度
    
    # 时间步进
    for i in range(1, num_steps):
        delta_t = dt
        a[i, :] = (F_i_t[i, :] - damping_ratio * v[i-1, :]) / (q * S)  # 考虑阻尼的加速度
        v[i, :] = v[i-1, :] + (1 - gamma) * delta_t * a[i-1, :] + gamma * delta_t * a[i, :]
        u[i, :] = u[i-1, :] + delta_t * v[i-1, :] + (0.5 - beta) * delta_t**2 * a[i-1, :] + beta * delta_t**2 * a[i, :]
    
    return u

# 计算风荷载
F_i_t = calculate_wind_load(U, rho, mu_s_i, A_i)

# 使用Newmark-beta算法计算位移，考虑阻尼
displacement = newmark_beta_damped(F_i_t, dt, total_time, damping_ratio)

# 将0号节点的位移设置为0
displacement[:, 0] = 0

# 计算位移的极大值
max_displacement = np.max(displacement, axis=0) * g

# 输出结果，以米为单位
print("各节点的结构位移的极大值 (米):")
for i in range(num_nodes):
    print(f"节点 {i}: {max_displacement[i]:.4f} 米")

各节点的结构位移的极大值 (米):
节点 0: 0.0000 米
节点 1: 19.1271 米
节点 2: 17.4447 米
节点 3: 17.3931 米
节点 4: 17.1762 米
节点 5: 17.2784 米
节点 6: 18.5116 米
节点 7: 18.9267 米
节点 8: 18.7782 米
节点 9: 19.0761 米
节点 10: 18.3899 米


In [ ]:
2.用时域方法计算节点最大位移的误差较大,可能方法不对